# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
pip install azureml-train-automl-runtime==1.20.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
from azureml.core import Workspace, Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml'

experiment=Experiment(ws, experiment_name)


Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RYJCRENRM to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster-4"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.data.dataset_factory import TabularDatasetFactory

csv_file = 'https://raw.githubusercontent.com/SwapnaKategaru/Project3/main/heart_failure_clinical_records_dataset.csv'

data = TabularDatasetFactory.from_delimited_files(csv_file)

In [6]:
%%writefile conda_dependencies.yml
dependencies:
- python=3.6.2
- pip:
  - scikit-learn=0.21.2
  - azureml-defaults

Writing conda_dependencies.yml


In [8]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
x, y = clean_data(data)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
train_data = pd.concat([x_train, y_train], axis = 1)

In [9]:
from azureml.train.automl import AutoMLConfig
import logging

automl_settings = {
    "enable_early_stopping" : True,
    "max_concurrent_iterations": 4,
    "primary_metric": 'AUC_weighted',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

automl_config = AutoMLConfig(
    experiment_timeout_minutes=15,
    task='classification',
    training_data=train_data,
    label_column_name='DEATH_EVENT',
    n_cross_validations=2)

In [12]:
from azureml.widgets import RunDetails
remote_run = experiment.submit(automl_config, show_output=True)
RunDetails(remote_run).show()

Running on local machine
Parent Run ID: AutoML_08081886-92ca-4154-8e50-048d8964b3d2

Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing f

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
best_run, fitted_model = remote_run.get_output()

In [26]:
best_run.get_file_names()

['accuracy_table',
 'confusion_matrix',
 'explanation/1607005c/classes.interpret.json',
 'explanation/1607005c/eval_data_viz.interpret.json',
 'explanation/1607005c/expected_values.interpret.json',
 'explanation/1607005c/features.interpret.json',
 'explanation/1607005c/global_names/0.interpret.json',
 'explanation/1607005c/global_rank/0.interpret.json',
 'explanation/1607005c/global_values/0.interpret.json',
 'explanation/1607005c/local_importance_values.interpret.json',
 'explanation/1607005c/per_class_names/0.interpret.json',
 'explanation/1607005c/per_class_rank/0.interpret.json',
 'explanation/1607005c/per_class_values/0.interpret.json',
 'explanation/1607005c/rich_metadata.interpret.json',
 'explanation/1607005c/visualization_dict.interpret.json',
 'explanation/800b3584/classes.interpret.json',
 'explanation/800b3584/eval_data_viz.interpret.json',
 'explanation/800b3584/expected_values.interpret.json',
 'explanation/800b3584/features.interpret.json',
 'explanation/800b3584/global_names/0.interpret.json',
 'explanation/800b3584/global_rank/0.interpret.json',
 'explanation/800b3584/global_values/0.interpret.json',
 'explanation/800b3584/local_importance_values.interpret.json',
 'explanation/800b3584/per_class_names/0.interpret.json',
 'explanation/800b3584/per_class_rank/0.interpret.json',
 'explanation/800b3584/per_class_values/0.interpret.json',
 'explanation/800b3584/rich_metadata.interpret.json',
 'explanation/800b3584/visualization_dict.interpret.json',
 'explanation/800b3584/ys_pred_proba_viz.interpret.json',
 'explanation/800b3584/ys_pred_viz.interpret.json',
 'outputs/conda_env_v_1_0_0.yml',
 'outputs/env_dependencies.json',
 'outputs/internal_cross_validated_models.pkl',
 'outputs/model.pkl',
 'outputs/pipeline_graph.json',
 'outputs/scoring_file_v_1_0_0.py']

In [18]:
best_run.get_file_names()[-3]

'outputs/model.pkl'

In [19]:
import pickle

In [20]:
best_run.download_file('outputs/model.pkl', './outputs/model.pkl')

In [50]:
joblib.load(filename='outputs/model.pkl')

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                    min_samples_split=0.10368421052631578,
                                                                                                    min_weight_fraction_leaf=0.0,
                                                                                                    n_estimators=25,
      

In [51]:
from azureml.core.model import Model
model = Model.register(model_name='automl-model', model_path='outputs/model.pkl', workspace=ws)

Registering model automl-model


In [52]:
import sklearn

from azureml.core.environment import Environment

environment = Environment("LocalDeploy")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn=={}".format(sklearn.__version__))

In [53]:
# %%writefile score.py

import json
import numpy as np
import pandas as pd
import os
import joblib
import pickle

def init():
    global model
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'outputs/model.pkl')
    print("Found model:", os.path.isfile(model_path)) #To check whether the model is actually present on the location we are looking at
    model = joblib.load(model_path)
def run(data):
    try:
        data = json.loads(data)['data']
        data = pd.DataFrame.from_dict(data)
        result = model.predict(data)
        return result.tolist()
    except Exception as e:
        error = str(e)
        return error

In [54]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-139107', subscription_id='3d1a56d2-7c81-4118-9790-f85d1acf0c77', resource_group='aml-quickstarts-139107'), name=automl-model, id=automl-model:2, version=2, tags={}, properties={})

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [55]:
from azureml.core.webservice import LocalWebservice
from azureml.core.model import InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=environment) 

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=1234)

local_service = Model.deploy(ws, "deeploy", [model], inference_config, deployment_config)

local_service.wait_for_deployment()

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 88619665ccdc418b86b5c2f0b6e8e5dd.azurecr.io
Logging into Docker registry 88619665ccdc418b86b5c2f0b6e8e5dd.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 88619665ccdc418b86b5c2f0b6e8e5dd.azurecr.io/azureml/azureml_8a90a8ae4f21dcd7231b9244082417af
 ---> 0f083f3495e8
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 231fc03ba1c0
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6IjNkMWE1NmQyLTdjODEtNDExOC05NzkwLWY4NWQxYWNmMGM3NyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTEzOTEwNyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTEzOTEwNyIsIndvcmtzcGFjZUlkIjoiODg2MTk2NjUtY2NkYy00MThiLTg2YjUtYzJmMGI2ZThlNWRkIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 41dcb43e49ba
 ---> 61972de0099d
Step 4/5 : RUN mv '/var/azureml-app/tmpgqf2touk.py' /var/azureml-app/main.py


ERROR:azureml._model_management._util:Error: Container has crashed. Did your init method fail?



WebserviceException: WebserviceException:
	Message: Error: Container has crashed. Did your init method fail?
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Error: Container has crashed. Did your init method fail?"
    }
}

In [ ]:
print('Local service port: {}'.format(local_service.port))

In [ ]:
import json

sample_input = json.dumps(
    {"data":
            [
                {
                    'age': 75.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 582,
                    'diabetes': 0,
                    'ejection_fraction': 20,
                    'high_blood_pressure': 1,
                    'platelets': 265000.0,
                    'serum_creatinine': 1.9,
                    'serum_sodium': 130,
                    'sex': 1,
                    'smoking': 0,
                    'time': 4
                },
                {
                    'age': 55.0,
                    'anaemia': 0,
                    'creatinine_phosphokinase': 7861,
                    'diabetes': 0,
                    'ejection_fraction': 38,
                    'high_blood_pressure': 0,
                    'platelets': 263358.03,
                    'serum_creatinine': 1.1,
                    'serum_sodium': 136,
                    'sex': 1,
                    'smoking': 0,
                    'time': 6
                }
            ] 
        }
)

local_service.run(sample_input)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service